In [2]:
import pandas as pd
import os
from tqdm import tqdm
from datetime import datetime

# ⚙️ Ajuste os caminhos conforme seu ambiente
pasta_dados = "/Users/juporto/dados_receita"
pasta_estab = os.path.join(pasta_dados, "estabelecimentos")
pasta_socios = os.path.join(pasta_dados, "socios")
pasta_empresas = os.path.join(pasta_dados, "empresas")

# 🧹 CNAEs e UFs alvo
# Lista de CNAEs no formato string com zeros à esquerda
cnaes_alvo = ["8513900"]  # Exemplo com 1 ou mais CNAEs
ufs_alvo = ["GO"]  # Lista vazia = todos os estados

# 📥 Função para ler os arquivos de sócios
def ler_socios(pasta_socios, cnpjs_filtrados):
    dfs = []
    arquivos = os.listdir(pasta_socios)
    for arq in tqdm(arquivos, desc="Sócios"):
        caminho = os.path.join(pasta_socios, arq)
        try:
            df = pd.read_csv(caminho, sep=';', header=None, encoding='latin1', dtype=str)
            df.columns = [
                "CNPJ_BASICO", "IDENTIFICADOR", "NOME", "CPF_CNPJ", "QUALIFICACAO", "DATA_ENTRADA",
                "PAIS", "REPRESENTANTE_NOME", "REPRESENTANTE_CPF", "REPRESENTANTE_QUALIFICACAO", "FAIXA_ETARIA"
            ]
            df = df[df["CNPJ_BASICO"].isin(cnpjs_filtrados)]
            dfs.append(df)
        except Exception as e:
            print(f"Erro em {arq}: {e}")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# 📥 Função para ler os arquivos de empresas (nome empresarial)
def ler_empresas(pasta_empresas):
    dfs = []
    arquivos = os.listdir(pasta_empresas)
    for arq in tqdm(arquivos, desc="Empresas"):
        caminho = os.path.join(pasta_empresas, arq)
        try:
            df = pd.read_csv(caminho, sep=';', header=None, encoding='latin1', dtype=str)
            df.columns = [
                "CNPJ_BASICO", "RAZAO_SOCIAL_NOME_EMPRESARIAL", "NATUREZA_JURIDICA",
                "QUALIFICACAO_RESPONSAVEL", "CAPITAL_SOCIAL", "PORTE_EMPRESA", "ENTE_FEDERATIVO_RESP"
            ]
            dfs.append(df)
        except Exception as e:
            print(f"Erro em {arq}: {e}")
    return pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()

# 📂 Ler e filtrar os estabelecimentos
dfs_estab = []
arquivos = os.listdir(pasta_estab)
for arq in tqdm(arquivos, desc="Estabelecimentos"):
    caminho = os.path.join(pasta_estab, arq)
    try:
        df = pd.read_csv(caminho, sep=';', header=None, encoding='latin1', dtype=str)
        df.columns = [
            "CNPJ_BASICO", "CNPJ_ORDEM", "CNPJ_DV", "MATRIZ_FILIAL",
            "NOME_FANTASIA", "SITUACAO_CADASTRAL", "DATA_SITUACAO_CADASTRAL",
            "MOTIVO_SITUACAO_CADASTRAL", "NOME_CIDADE_EXTERIOR", "PAIS",
            "DATA_INICIO_ATIVIDADE", "CNAE_PRINCIPAL", "CNAE_SECUNDARIO",
            "TIPO_LOGRADOURO", "LOGRADOURO", "NUMERO", "COMPLEMENTO", "BAIRRO",
            "CEP", "UF", "MUNICIPIO", "DDD1", "TELEFONE1", "DDD2", "TELEFONE2",
            "DDD_FAX", "FAX", "EMAIL", "SITUACAO_ESPECIAL", "DATA_SITUACAO_ESPECIAL"
        ]

        # 🔍 Filtragem por CNAE e UF (agora com listas)
        df_filtrado = df[df["CNAE_PRINCIPAL"].isin(cnaes_alvo)]
        if ufs_alvo:  # só aplica se lista não estiver vazia
            df_filtrado = df_filtrado[df_filtrado["UF"].isin(ufs_alvo)]

        if not df_filtrado.empty:
            dfs_estab.append(df_filtrado)

    except Exception as e:
        print(f"Erro em {arq}: {e}")

df_estab = pd.concat(dfs_estab, ignore_index=True) if dfs_estab else pd.DataFrame()

# 🛠 Corrigir CNPJ_BASICO
if not df_estab.empty:
    df_estab['CNPJ_BASICO'] = df_estab['CNPJ_BASICO'].astype(str).str.zfill(8)

    # ✅ 🔄 Substituir código do município pelo nome
    df_tom = pd.read_csv("municipios.csv", sep=';', encoding='latin1')
    df_tom = df_tom.rename(columns={
        "CÓDIGO DO MUNICÍPIO - TOM": "codigo_municipio",
        "MUNICÍPIO - TOM": "municipio_nome"
    })
    df_tom["codigo_municipio"] = df_tom["codigo_municipio"].astype(str)

    df_estab["codigo_municipio"] = df_estab["MUNICIPIO"].astype(str).str.zfill(4)
    df_estab = df_estab.merge(
        df_tom[["codigo_municipio", "municipio_nome"]],
        on="codigo_municipio",
        how="left"
    )
    df_estab["MUNICIPIO"] = df_estab["municipio_nome"]
    df_estab.drop(columns=["municipio_nome", "codigo_municipio"], inplace=True)

    # 🔹 Sócios
    cnpjs = df_estab["CNPJ_BASICO"].unique()
    df_socios = ler_socios(pasta_socios, cnpjs)
    df_socios['CNPJ_BASICO'] = df_socios['CNPJ_BASICO'].astype(str).str.zfill(8)

    faixa_map = {
        "1": "Menor de 16 anos",
        "2": "16 a 20 anos",
        "3": "21 a 30 anos",
        "4": "31 a 40 anos",
        "5": "41 a 50 anos",
        "6": "51 a 60 anos",
        "7": "61 a 70 anos",
        "8": "71 a 80 anos",
        "9": "Mais de 80 anos"
    }
    df_socios["FAIXA_ETARIA"] = df_socios["FAIXA_ETARIA"].replace(faixa_map)

    # 🔹 Empresas
    df_empresas = ler_empresas(pasta_empresas)
    df_empresas['CNPJ_BASICO'] = df_empresas['CNPJ_BASICO'].astype(str).str.zfill(8)
else:
    df_socios = pd.DataFrame()
    df_empresas = pd.DataFrame()

# 🔀 Merge: estabelecimentos + empresas + sócios
df_merged = pd.merge(df_estab, df_empresas, on="CNPJ_BASICO", how="left")
df_merged = pd.merge(df_merged, df_socios, on="CNPJ_BASICO", how="left")


# 💾 Gera nome único com data e hora (ex: empresas_socios_nome_razao_2025-11-09_1430.csv)
timestamp = datetime.now().strftime("%Y-%m-%d_%H%M")
nome_arquivo = f"empresas_socios_nome_razao_{timestamp}.csv"

# Junta todos os CNAEs da lista em uma string separada por "_"
cnaes_str = "_".join(cnaes_alvo) if cnaes_alvo else "todos_CNAEs"

# Monta o nome final do arquivo (ex: empresas_socios_nome_razao_CNAE_8513900_2025-11-09_1430.csv)
nome_arquivo = f"empresas_socios_nome_razao_CNAE_{cnaes_str}_{timestamp}.csv"

df_merged.to_csv(nome_arquivo, index=False, sep=';', encoding='utf-8-sig')

print(f"📁 Arquivo '{nome_arquivo}' exportado com sucesso!")


Empresas: 100%|█████████████████████████████████| 10/10 [00:58<00:00,  5.83s/it]


📁 Arquivo 'empresas_socios_nome_razao_CNAE_8513900_2025-11-09_2131.csv' exportado com sucesso!
